# Create training dataset with Label-Maker for detecting building footprint

In this usecase, we will create a training dataset for detecting building footprint of Dar es Salaam with Label Maker. We will then train a neural net for building segmentation.

Before the taining dataset creation, I recommend you go throught these steps. 

- [setup a python virtual environment](https://gist.github.com/wronk/a902185f5f8ed018263d828e1027009b); I use python 3.6.3.
- [install Tippecanoe](https://github.com/mapbox/tippecanoe) and [pip install label maker](https://github.com/developmentseed/label-maker);
- `pip install jupyter`;
- [Register and get Mapbox access token](https://www.mapbox.com/help/how-access-tokens-work/)
- run this python notebook and follow allowing the following instruction.

In [ ]:
%%file config.json 
{
  "country": "united_republic_of_tanzania",
  "bounding_box": [39.22222137451172,-6.8209322101709935,39.28736686706543,-6.776955027395981],
  "zoom": 17,
  "classes": [
    { "name": "Buildings", "filter": ["has", "building"] , "buffer": 2}
  ],
  "imagery": "https://a.tiles.mapbox.com/v4/digitalglobe.2lnpeioh/{z}/{x}/{y}.jpg?access_token=TOKEN"
  "ml_type": "segmentation"
}

In [ ]:
!label-maker download --dest tz_buildings --config config.json

In [ ]:
!label-maker labels --dest tz_buildings --config config.json

In [ ]:
!label-maker images --dest tz_buildings --config config.json

# Move to a GPU cloud computer

This example was made from AWS P2.xlarge machine, it's an **Ubuntu Deep Learning AMI** for specific. 

I did these following steps to be able to start this notebook in the cloud computer with `Nvidia-Docker`.

- 1. install [docker](https://docs.aws.amazon.com/AmazonECS/latest/developerguide/docker-basics.html) and [nvidia-docker](https://towardsdatascience.com/using-docker-to-set-up-a-deep-learning-environment-on-aws-6af37a78c551) on AWS EC2 p2.xlarge instance;
- 2. running `git clone https://github.com/Geoyi/FOSS_4g_Pixel_decoder.git` and `cd FOSS_4g_Pixel_decoder`;
- 3. built the nvidia-docker image to run the notebook, by running `docker build -t pixel-decoder-notebook .`, it will create a nvidia-docker called `pixel-decoder-notebook`;
- 4. incide the `FOSS_4g_Pixel_decoder` directory, run `git clone https://github.com/Geoyi/pixel-decoder.git`, and move the notebook `Label-Maker_Pixel-Decoder.ipynb` to `pixel-decoder` directory;
- 5. `cd pixel-decoder` and run `nvidia-docker run -v $PWD:/example -p 8888:8888 -it pixel-decoder-notebook`;
- 6. in another terminal tag, log in and active to your notebook by runnin `ssh -i Your_AWS.pem -L 8888:localhost:8888 ubuntu@ec2-XX-XXX-XXX-XXX.compute-1.amazonaws.com`;
- log in to your notebook in your browser by runnin `localhost:8888/?token=xxxxxx`;

Now you are ready to run next steps.


In [ ]:
!pip install -e .

In [ ]:
!pixel_decoder train --batch_size=16 \
                    --imgs_folder=tiles \
                    --masks_folder=new_labels \
                    --models_folder=tz_building_model \
                    --model_id=resnet_unet \
                    --origin_shape_no=256 \
                    --border_no=32

Using TensorFlow backend.
2018-08-28 19:25:01.063305: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-08-28 19:25:01.139122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:897] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-08-28 19:25:01.139526: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1405] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:1e.0
totalMemory: 11.17GiB freeMemory: 11.09GiB
2018-08-28 19:25:01.139563: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1484] Adding visible gpu devices: 0
2018-08-28 19:25:01.438861: I tensorflow/core/common_runtime/gpu/gpu_device.cc:965] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-08-28 19:25:01.438927: I tensorflow/core/common_runtime/gpu/gpu_d

Epoch 1/25
 - 34s - loss: 0.9150 - dice_coef: 3.5905e-06 - dice_coef_rounded: 6.2873e-04 - binary_crossentropy: 0.4331 - val_loss: 0.8748 - val_dice_coef: 6.3028e-06 - val_dice_coef_rounded: 0.6944 - val_binary_crossentropy: 0.1654
Epoch 2/25
 - 16s - loss: 0.8615 - dice_coef: 1.4520e-05 - dice_coef_rounded: 0.7256 - binary_crossentropy: 0.0768 - val_loss: 0.8575 - val_dice_coef: 1.9712e-05 - val_dice_coef_rounded: 1.0000 - val_binary_crossentropy: 0.0498
Epoch 3/25
 - 16s - loss: 0.8550 - dice_coef: 2.9771e-05 - dice_coef_rounded: 1.0000 - binary_crossentropy: 0.0336 - val_loss: 0.8541 - val_dice_coef: 3.4951e-05 - val_dice_coef_rounded: 1.0000 - val_binary_crossentropy: 0.0278
Epoch 4/25
 - 17s - loss: 0.8534 - dice_coef: 4.3279e-05 - dice_coef_rounded: 1.0000 - binary_crossentropy: 0.0226 - val_loss: 0.8528 - val_dice_coef: 5.1862e-05 - val_dice_coef_rounded: 1.0000 - val_binary_crossentropy: 0.0186
Epoch 5/25
 - 17s - loss: 0.8526 - dice_coef: 5.5904e-05 - dice_coef_rounded: 1.0000

In [ ]:
!pixel_decoder predict --imgs_folder=tiles\
                    --test_folder=tiles \
                    --models_folder=tz_building_model \
                    --pred_folder=predictions-tz_building \
                    --model_id=resnet_unet \
                    --origin_shape_no=256 \
                    --border_no=32

In [ ]:
import os, glob
import numpy as np
from os import path as op
import matplotlib.pyplot as plt
%matplotlib inline
import skimage.io

In [ ]:
tiles =sorted(glob.glob("tiles/*.jpg"))
labels =sorted(glob.glob("labels/*.png"))
preds= sorted(glob.glob("predictions-tz_building/resnet_unet/*.png"))
def grey2rgb(img):
    """
    Create model prediction result to 3 channel, so we can see the result in black and white.
    """
    new_img = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            new_img.append([img[i][j]]*3)
    new_img = np.array(new_img).reshape(img.shape[0], img.shape[1], 3)
    return new_img

In [ ]:
pred_list = [op.basename(f).split(".")[0] for f in preds]

In [ ]:
for pred in pred_list:
    img = skimage.io.imread("tiles/{}.jpg".format(pred))
    mask = skimage.io.imread("labels/{}.png".format(pred))
    pred_mask = grey2rgb(skimage.io.imread("predictions-tz_building/resnet_unet/{}.png".format(pred)))
    masked = np.ma.masked_where(pred_mask >=1, pred_mask)

    fig, ax =  plt.subplots(1, 3, figsize=(30, 10))
    ax[-3].imshow(img, 'gray', interpolation='none')
    ax[-3].set_title('Origin RGB image Tile')

    ax[-2].imshow(img, 'gray', interpolation='none')
    ax[-2].imshow(mask, 'jet', interpolation='none', alpha=0.6)
    ax[-2].set_title('Ground-truth over image')

    ax[-1].imshow(img, 'gray', interpolation='none')
    ax[-1].imshow(masked, 'jet', interpolation='none', alpha=0.6)
    ax[-1].set_title('Predition over images')

    fig.savefig("tz_buildings_pred_{}.png".format(pred))

In [ ]:
for pred in pred_list:
    pred_mask = grey2rgb(skimage.io.imread("predictions-tz_building/resnet_unet/{}.png".format(pred)))
    plt.imshow(pred_mask)